In [1]:
import os
import math
from itertools import combinations
import pandas as pd

In [43]:
wd = "/Users/timvigers/Documents/Work/Erin Cobry/Nocturnal Alarms/"
# Subject information and dates
subject_dates = pd.read_csv(wd+"Data_Clean/subject_dates.csv")
# Dictionary for results
dict={'File': [], 'Nights': [], 'Total Alarms': [], 'Threshold Alarms': [], 'Maintenance Alarms': [], \
    'HCL Alarms': [], 'Pump Alarms': [], 'Other Alarms': []}
# Columns to read in
fields = ['First Name','Last Name','Patient ID','Unnamed: 16']

In [179]:
# Testing cell
file = os.listdir(wd+"Data_Raw/CSVs/")[0]
df = pd.read_csv(wd+"Data_Raw/CSVs/"+file,usecols=fields,dtype={'Patient ID':'str','Unnamed: 16':'str'})
t = file.split(" ")[1]
t_cols = [col for col in subject_dates.columns if t.lower() in col]
start = pd.to_datetime(subject_dates.loc[ind,t_cols[0]],format="%m/%d/%y").dt.date.reset_index(drop=True)
end = pd.to_datetime(subject_dates.loc[ind,t_cols[1]],format="%m/%d/%y").dt.date.reset_index(drop=True)

df.columns = df.loc[5,]
df = df.drop(list(range(0,6)),axis = 0).reset_index(drop=True)
    
df['Date'] = pd.to_datetime(df['Date'],format="%m/%d/%y",errors="coerce")
df = df.set_index('Date').sort_index()
df = df.loc[str(start.loc[0]):str(end.loc[0])]

5,Index,Time,Alarm
Date,,,
2018-11-15,650,0:00:00,NaN
2018-11-15,329,4:30:00,NaN
2018-11-15,328,6:39:57,NaN
2018-11-15,327,6:45:22,NaN
2018-11-15,326,7:00:00,NaN
...,...,...,...
2018-11-29,104,19:00:00,NaN
2018-11-29,103,19:00:00,NaN
2018-11-29,112,4:30:00,NaN


In [49]:
# Iterate through files in wd
for file in os.listdir(wd+"Data_Raw/CSVs/"):
    df = pd.read_csv(wd+"Data_Raw/CSVs/"+file,usecols=fields,dtype={'Patient ID':'str','Unnamed: 16':'str'})
    # Check name
    first_name = str(df.loc[0,'First Name']).lower()
    last_name = str(df.loc[0,'Last Name']).lower()
    ind = subject_dates['name'].str.lower() == first_name + ' ' + last_name
    if not any(ind):
        print("The name '" + first_name + ' ' + last_name + "' could not be found")
        print(file)
    # Get correct dates for timepoint
    t = file.split(" ")[1]
    t_cols = [col for col in subject_dates.columns if t.lower() in col]
    start = pd.to_datetime(subject_dates.loc[ind,t_cols[0]],format="%m/%d/%y").dt.date.reset_index(drop=True)
    end = pd.to_datetime(subject_dates.loc[ind,t_cols[1]],format="%m/%d/%y").dt.date.reset_index(drop=True)
    # Rename columns
    df.columns = df.loc[5,]
    df = df.drop(list(range(0,6)),axis = 0)
    # Filter out unnecessary days
    df['Date'] = pd.to_datetime(df['Date'],format="%m/%d/%y",errors="coerce")
    df = df.set_index('Date').sort_index()
    df = df.loc[str(start.loc[0]):str(end.loc[0])]
    # Count number of nights
    
    # Pull all alarms between 10p-6a 
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'],format="%m/%d/%y %H:%M:%S",errors="coerce")
    df['Time'] = pd.to_datetime(df['Time'],format="%H:%M:%S",errors="coerce")
    # Index by time
    df = df.set_index('Time')
    all_alarms = df['Alarm'].between_time("22:00:00","6:00:00",include_end=False).dropna().str.lower()
    dates = df.loc[all_alarms.index,'Datetime']
    # Don't count the alarm if it includes the words:  QUIET, BOLUS, ENTER BG, 
    matches = ["quiet","bolus","enter bg"]
    all_alarms = [alarm for alarm in all_alarms if all(x not in alarm for x in matches)]
    # Alarm types (per Cari)
    # Threshold
    threshold = [alarm for alarm in all_alarms if 'alert' in alarm]
    threshold = [alarm for alarm in threshold if 'cal' not in alarm]
    # Maintenance
    matches = ["sensor","cal","signal","transmtr"]
    maintenance = [alarm for alarm in all_alarms if any(x in alarm for x in matches)]
    maintenance = [alarm for alarm in maintenance if 'auto mode' not in alarm]
    # HCL specific
    hcl = [alarm for alarm in all_alarms if 'auto' in alarm]
    # Pump maintenance
    matches = ["pump","battery","reservoir"]
    pump = [alarm for alarm in all_alarms if any(x in alarm for x in matches)]
    # Other
    other = [alarm for alarm in all_alarms if 'other' in alarm]
    # Return
    dict['File'].append(file.replace(".csv","")) 
    dict['Total Alarms'].append(len(all_alarms)) 
    dict['Threshold Alarms'].append(len(threshold)) 
    dict['Maintenance Alarms'].append(len(maintenance)) 
    dict['HCL Alarms'].append(len(hcl)) 
    dict['Pump Alarms'].append(len(pump)) 
    dict['Other Alarms'].append(len(other))

In [51]:
t

'T4'

In [46]:
help(grep)

NameError: name 'grep' is not defined